In [1]:
import os

# necessay data modelling
import time
from datetime import datetime
from datetime import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# advance stats and machine learning
##import tensorflow as tf
import statsmodels.api as sm
import random
import scipy.stats as stats
from scipy.stats import norm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from scipy.stats import skew, kurtosis, mode

## Helper Function

In [2]:
def detect_outliers(df1, column):
    # First, ensure the column is numeric
    if df1[column].dtype == 'object':
        try:
            # Try to convert to numeric, with errors='coerce' to convert non-convertible values to NaN
            df1[column] = pd.to_numeric(df1[column], errors='coerce')
        except:
            # If conversion fails completely, return zeros to avoid breaking the loop
            return 0, 0, 0

    # Drop NaN values before calculating quantiles
    valid_data = df1[column].dropna()

    # Check if we have enough data after dropping NaN values
    if len(valid_data) < 2:
        return 0, 0, 0

    Q1 = valid_data.quantile(0.25)
    Q3 = valid_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df1[(df1[column] < lower_bound) | (df1[column] > upper_bound)]
    valid_values = len(df1[column]) - df1[column].isna().sum()
    return outliers.shape[0], lower_bound, upper_bound, valid_values

In [3]:
# Define the function
def generate_numerical_profile(df):
    numerical_df = df.select_dtypes(include=[np.number])
    summary = []

    for col in numerical_df.columns:
        series = numerical_df[col].dropna()
        if series.empty:
            continue

        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = series[(series < lower_bound) | (series > upper_bound)]

        try:
          mode_result = mode(series, nan_policy='omit')
          mode_val = mode_result.mode[0] if len(mode_result.mode) > 0 else np.nan
        except:
          mode_val = np.nan

        summary.append({
            "Column name": col,
            "no. of values": len(series),
            "no. of nulls": df[col].isnull().sum(),
            "null %": round(df[col].isnull().mean() * 100, 2),
            "min": series.min(),
            "max": series.max(),
            "range": series.max() - series.min(),
            "avg": series.mean(),
            "std.dev": series.std(),
            "median": series.median(),
            "25 percentile": q1,
            "75 percentile": q3,
            "distinct values": series.nunique(),
            "Probable outliers (use IQR)": len(outliers),
            "skewness": skew(series),
            "kurtosis": kurtosis(series),
            "mode": mode_val,
            "zero count": (series == 0).sum()
        })

    return pd.DataFrame(summary)

In [4]:
def generate_categorical_profile(df):
    categorical_df = df.select_dtypes(include=['object', 'category'])
    summary = []

    for col in categorical_df.columns:
        series = categorical_df[col]
        value_counts = series.value_counts(dropna=False).to_dict()
        unique_categories = list(series.dropna().unique())
        num_categories = len(unique_categories)
        top_3 = list(series.value_counts().head(3).index)
        missing_pct = round(series.isnull().mean() * 100, 2)
        is_binary = num_categories == 2

        summary.append({
            "Column Name": col,
            "Number of Categories": num_categories,
            "Category List": unique_categories,
            "Value Counts (dict)": value_counts,
            "Top 3 Categories": top_3,
            "Missing %": missing_pct,
            "Is Binary": is_binary
        })

    return pd.DataFrame(summary)

In [5]:
# Replacing nulls

def fill_nulls(df, col, method='mean', custom_value=None):
    if method == 'mean':
        val = df[col].mean()
    elif method == 'median':
        val = df[col].median()
    elif method == 'mode':
        val = df[col].mode()[0]
    elif method == 'custom':
        val = custom_value
    else:
        raise ValueError("Method must be 'mean', 'median', 'mode', or 'custom'")

    df[col] = df[col].fillna(val)
    return df

In [6]:
def convert_dtype(df, col, dtype):
    df[col] = df[col].astype(dtype)
    return df

In [7]:
def parse_dates(df, col, date_format=None):
    df[col] = pd.to_datetime(df[col], format=date_format, errors='coerce')
    return df

In [8]:
def standardize_text(df, col):
    df[col] = df[col].str.strip().str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)
    return df

In [9]:
def trim_whitespace(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()
    return df

In [10]:
def detect_duplicates(df):
    return df[df.duplicated()]

In [11]:
def one_hot_encode_smart(df, column, drop_least_frequent=True, prefix=None):
    """
    Performs one-hot encoding on a categorical column while dropping one category
    (default: the least frequent) to avoid multicollinearity.

    Parameters:
        df (DataFrame): Input DataFrame.
        column (str): Column to encode.
        drop_least_frequent (bool): If True, drops the least frequent category. If False, drops the first by default.
        prefix (str): Optional prefix for encoded columns.

    Returns:
        DataFrame: DataFrame with one-hot encoded columns and the original column dropped.
    """
    freqs = df[column].value_counts(dropna=False)
    drop_cat = freqs.idxmin() if drop_least_frequent else freqs.index[0]

    dummies = pd.get_dummies(df[column], prefix=prefix or column, drop_first=False)

    if drop_cat in dummies.columns:
        dummies.drop(columns=[drop_cat], inplace=True)

    df = df.drop(columns=[column]).join(dummies)
    return df

In [12]:
def scale_features(
    df,
    columns,
    method='standard',
    return_scaler=False,
    add_suffix=True
):
    """
    Scales numerical columns using standard, min-max, or robust scaling.

    Parameters:
        df (DataFrame): Input DataFrame.
        columns (list): Columns to scale.
        method (str): 'standard', 'minmax', or 'robust'.
        return_scaler (bool): Return scaler object.
        add_suffix (bool): If True, keep original and add new scaled columns with '_scaled' suffix.

    Returns:
        DataFrame (and scaler if return_scaler=True)
    """
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

    scaler_map = {
        'standard': StandardScaler(),
        'minmax': MinMaxScaler(),
        'robust': RobustScaler()
    }

    if method not in scaler_map:
        raise ValueError("Method must be 'standard', 'minmax', or 'robust'")

    scaler = scaler_map[method]
    df_scaled = df.copy()

    scaled_values = scaler.fit_transform(df_scaled[columns])

    if add_suffix:
        for i, col in enumerate(columns):
            df_scaled[col + '_scaled'] = scaled_values[:, i]
    else:
        df_scaled[columns] = scaled_values

    return (df_scaled, scaler) if return_scaler else df_scaled

In [38]:
def find_non_numeric_values(df, column_name):
    """
    Identifies non-numeric values in a pandas DataFrame column.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the column to check
    column_name : str
        The name of the column to check for non-numeric values
        
    Returns:
    --------
    dict
        A dictionary with non-numeric values as keys and their count as values
    """
    import pandas as pd
    import numpy as np
    
    # Make a copy of the column to avoid modifying the original
    series = df[column_name].copy()
    
    # Try to convert to numeric, coerce errors to NaN
    numeric_series = pd.to_numeric(series, errors='coerce')
    
    # Find the indices where conversion produced NaN (these are non-numeric values)
    problematic_indices = series[numeric_series.isna()].index
    
    # Get the original values at these indices
    problematic_values = series[problematic_indices].value_counts().to_dict()
    
    return problematic_values

## File - Read

In [13]:
os.chdir('/Users/vivan/Desktop/Central File Manager/03 Projects/03 Datathon/genderwagegap/01 Data')

In [14]:
os.listdir()

['data_sample.csv', 'data_sample.csv.zip', 'IND_Codes.csv', 'OCCSOC_Codes.csv']

In [15]:
data = pd.read_csv('data_sample.csv', encoding='latin1')

/var/folders/l5/9tzh8xvn4c9bb2qc2r3pkqtc0000gn/T/ipykernel_777/436843986.py:1: DtypeWarning: Columns (5,15,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_sample.csv', encoding='latin1')


In [16]:
pd.set_option('display.max_columns', None)

In [17]:
data

,REGION,STATEFIP,PERWT,NCHILD,SEX,AGE,MARST,MARRNO,RACE,RACED,BPL,BPLD,ANCESTR1,ANCESTR1D,CITIZEN,YRNATUR,LANGUAGE,LANGUAGED,SPEAKENG,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,CLASSWKR,CLASSWKRD,OCCSOC,IND,WKSWORK1,UHRSWORK,INCTOT,INCWAGE,INCINVST,INCOTHER,INCEARN,TRANWORK,TRANTIME,DEPARTS,ARRIVES
0,East South Central Div.,Alabama,48.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909
1,East South Central Div.,Alabama,9.0,0 children present,Female,52,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",537051,6390,52,40,40600,40600,0,0,40600,"Auto, truck, or van",5,802,809
2,East South Central Div.,Alabama,46.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,State govt employee,412010,5391,52,41,6000,6000,0,0,6000,"Auto, truck, or van",25,532,559
3,East South Central Div.,Alabama,54.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909
4,East South Central Div.,Alabama,42.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Florida,Florida,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",352010,8680,52,35,26400,26400,0,0,26400,Walked only,3,1455,1459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502358,Mountain Division,Wyoming,155.0,2,Male,32,"Married, spouse present",Married once,White,White,Arizona,Arizona,Not Reported,Not Reported,NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",4750YY,380,52,40,70000,70000,0,0,70000,"Auto, truck, or van",45,702,749
502359,Mountain Division,Wyoming,186.0,0 children present,Male,23,Never married/single,Not Applicable,White,White,Wyoming,Wyoming,Spanish,Spanish,NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",533030,6170,52,50,20000,20000,0,0,20000,"Auto, truck, or van",20,602,624
502360,Mountain Division,Wyoming,152.0,0 children present,Male,46,"Married, spouse present",Married once,White,White,Washington,Washington,White/Caucasian,"White/Caucasian (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,GED or alternative credential,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",471011,490,52,80,95000,95000,0,0,95000,"Auto, truck, 

In [18]:
ind_codes = pd.read_csv('IND_Codes.csv')

In [19]:
ind_codes.head()

,IND Codes,Industry,Category,Sub Category
0,170,Crop production,"Agriculture, Forestry, Fishing, and Hunting, a...","Agriculture, Forestry, Fishing, and Hunting"
1,180,Animal production and aquaculture,"Agriculture, Forestry, Fishing, and Hunting, a...","Agriculture, Forestry, Fishing, and Hunting"
2,190,Forestry except logging,"Agriculture, Forestry, Fishing, and Hunting, a...","Agriculture, Forestry, Fishing, and Hunting"
3,270,Logging,"Agriculture, Forestry, Fishing, and Hunting, a...","Agriculture, Forestry, Fishing, and Hunting"
4,280,"Fishing, hunting and trapping","Agriculture, Forestry, Fishing, and Hunting, a...","Agriculture, Forestry, Fishing, and Hunting"


In [20]:
occsoc_codes = pd.read_csv('OCCSOC_Codes.csv')

## Merge Dataset

In [21]:
data_ind = pd.merge(
    left=data,
    right=ind_codes,
    left_on='IND',
    right_on='IND Codes',
    how='left'
)

In [22]:
data_ind

,REGION,STATEFIP,PERWT,NCHILD,SEX,AGE,MARST,MARRNO,RACE,RACED,BPL,BPLD,ANCESTR1,ANCESTR1D,CITIZEN,YRNATUR,LANGUAGE,LANGUAGED,SPEAKENG,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,CLASSWKR,CLASSWKRD,OCCSOC,IND,WKSWORK1,UHRSWORK,INCTOT,INCWAGE,INCINVST,INCOTHER,INCEARN,TRANWORK,TRANTIME,DEPARTS,ARRIVES,IND Codes,Industry,Category,Sub Category
0,East South Central Div.,Alabama,48.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schoo...","Educational Services, and Health Care and Soci...",Educational Services
1,East South Central Div.,Alabama,9.0,0 children present,Female,52,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",537051,6390,52,40,40600,40600,0,0,40600,"Auto, truck, or van",5,802,809,6390,Warehousing and storage,"Transportation and Warehousing, and Utilities ...","Transportation and Warehousing, and Utilities ..."
2,East South Central Div.,Alabama,46.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,State govt employee,412010,5391,52,41,6000,6000,0,0,6000,"Auto, truck, or van",25,532,559,5391,"General merchandise stores, including warehous...",Retail Trade,Retail Trade
3,East South Central Div.,Alabama,54.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schoo...","Educational Services, and Health Care and Soci...",Educational Services
4,East South Central Div.,Alabama,42.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Florida,Florida,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",352010,8680,52,35,26400,26400,0,0,26400,Walked only,3,1455,1459,8680,Restaurants and other food services,"Arts, Entertainment, and Recreation, and Accom...",Accommodation and Food Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502358,Mountain Division,Wyoming,155.0,2,Male,32,"Married, spouse present",Married once,White,White,Arizona,Arizona,Not Reported,Not Reported,NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",4750YY,380,52,40,70000,70000,0,0,70000,"Auto, truck, or van",45,702,749,380,Coal mining,"Agriculture, Forestry, Fishing, and Hunting, a...","Mining, Quarrying, and Oil and Gas Extraction"
502359,Mountain 

In [23]:
occsoc_codes.head()

,Main Category,Sub Category,2018 Onward ACS/PRCS,Occupation title
0,Not Applicable,Not Applicable,0,Not Applicable (Under 16 years or not in the l...
1,"MANAGEMENT, BUSINESS, SCIENCE, AND ARTS OCCUPA...",Management Occupations:,1110XX,Chief Executives and Legislators
2,"MANAGEMENT, BUSINESS, SCIENCE, AND ARTS OCCUPA...",Management Occupations:,111021,General and Operations Managers
3,"MANAGEMENT, BUSINESS, SCIENCE, AND ARTS OCCUPA...",Management Occupations:,112011,Advertising and Promotions Managers
4,"MANAGEMENT, BUSINESS, SCIENCE, AND ARTS OCCUPA...",Management Occupations:,112021,Marketing Managers


In [24]:
occsoc_codes = occsoc_codes.rename(columns={'Sub Category':'OCCSOC Sub Category'})

In [25]:
data_final = pd.merge(
    left=data_ind,
    right=occsoc_codes,
    left_on='OCCSOC',
    right_on='2018 Onward ACS/PRCS',
    how='left'
)

In [26]:
data_final

,REGION,STATEFIP,PERWT,NCHILD,SEX,AGE,MARST,MARRNO,RACE,RACED,BPL,BPLD,ANCESTR1,ANCESTR1D,CITIZEN,YRNATUR,LANGUAGE,LANGUAGED,SPEAKENG,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,CLASSWKR,CLASSWKRD,OCCSOC,IND,WKSWORK1,UHRSWORK,INCTOT,INCWAGE,INCINVST,INCOTHER,INCEARN,TRANWORK,TRANTIME,DEPARTS,ARRIVES,IND Codes,Industry,Category,Sub Category,Main Category,OCCSOC Sub Category,2018 Onward ACS/PRCS,Occupation title
0,East South Central Div.,Alabama,48.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schoo...","Educational Services, and Health Care and Soci...",Educational Services,SERVICE OCCUPATIONS:,Personal Care and Service Occupations:,399041,Residential Advisors
1,East South Central Div.,Alabama,9.0,0 children present,Female,52,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",537051,6390,52,40,40600,40600,0,0,40600,"Auto, truck, or van",5,802,809,6390,Warehousing and storage,"Transportation and Warehousing, and Utilities ...","Transportation and Warehousing, and Utilities ...","PRODUCTION, TRANSPORTATION, AND MATERIAL MOVIN...",Material Moving Occupations:,537051,Industrial Truck and Tractor Operators
2,East South Central Div.,Alabama,46.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,State govt employee,412010,5391,52,41,6000,6000,0,0,6000,"Auto, truck, or van",25,532,559,5391,"General merchandise stores, including warehous...",Retail Trade,Retail Trade,SALES AND OFFICE OCCUPATIONS:,Sales and Related Occupations:,412010,Cashiers
3,East South Central Div.,Alabama,54.0,0 children present,Male,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schoo...","Educational Services, and Health Care and Soci...",Educational Services,SERVICE OCCUPATIONS:,Personal Care and Service Occupations:,399041,Residential Advisors
4,East South Central Div.,Alabama,42.0,0 children present,Female,18,Never married/single,Not Applicable,Black/African American,Black/African American,Florida,Florida,African-American,"African-American (1990-2000, ACS, PRCS)",NaN,NaN,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",352010,8680,52,35,26400,26400,0,0,26400,Walked only,3,1455,1459,8680,Restaurants and other food services,"Arts, Entertainment, and Recreation, and Accom...",Accommodation and Food Services,SERVICE OCCUPATIONS:,Food Preparation and Serving Related Occupations:,352010,Cooks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [27]:
generate_numerical_profile(data_final)

,Column name,no. of values,no. of nulls,null %,min,max,range,avg,std.dev,median,25 percentile,75 percentile,distinct values,Probable outliers (use IQR),skewness,kurtosis,mode,zero count
0,PERWT,502363,0,0.0,1.0,2471.0,2470.0,107.669132,92.227745,81.0,53.0,131.0,1055,37065,2.928824,16.364742,NaN,0
1,IND,502363,0,0.0,170.0,9590.0,9420.0,6175.757974,2667.678290,7270.0,4580.0,8090.0,262,0,-0.899981,-0.383716,NaN,0
2,WKSWORK1,502363,0,0.0,50.0,52.0,2.0,51.980297,0.190678,52.0,52.0,52.0,3,5617,-9.906172,97.997289,NaN,0
3,INCTOT,502363,0,0.0,-9500.0,1487400.0,1496900.0,80917.390530,87420.436979,58000.0,36000.0,92500.0,11356,35337,4.223834,24.963733,NaN,0
4,INCWAGE,502363,0,0.0,0.0,787000.0,787000.0,73129.947046,78261.462905,53000.0,33000.0,86000.0,935,32606,3.970074,21.287865,NaN,20553
5,INCINVST,502363,0,0.0,-6300.0,522000.0,528300.0,1772.900739,17421.416029,0.0,0.0,0.0,695,56589,17.178026,333.837477,NaN,445774
6,INCOTHER,502363,0,0.0,0.0,76000.0,76000.0,352.587153,2869.488204,0.0,0.0,0.0,500,17563,13.277625,230.499116,NaN,484800
7,TRANTIME,502363,0,0.0,0.0,163.0,163.0,21.224226,22.362510,15.0,5.0,30.0,141,15059,2.318866,8.544759,NaN,101092
8,DEPARTS,502363,0,0.0,0.0,2345.0,2345.0,627.603430,433.007743,702.0,502.0,802.0,151,139020,0.711969,1.820294,NaN,101092
9,ARRIVES,502363,0,0.0,0.0,2359.0,2359.0,657.883620,442.539338,719.0,519.0,824.0,286,139438,0.572473,1.540248,NaN,101092


In [28]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = None
pd.set_option('max_colwidth', None)

In [29]:
generate_categorical_profile(data_final)

Column Name  Number of Categories  \
0                 REGION                     9   
1               STATEFIP                    51   
2                 NCHILD                    10   
3                    SEX                     2   
4                    AGE                   158   
5                  MARST                     6   
6                 MARRNO                     4   
7                   RACE                     9   
8                  RACED                   139   
9                    BPL                   123   
10                  BPLD                   219   
11              ANCESTR1                   206   
12             ANCESTR1D                   233   
13               CITIZEN                     3   
14               YRNATUR                   144   
15              LANGUAGE                    63   
16             LANGUAGED                   115   
17              SPEAKENG                     5   
18                  EDUC                    11   
19                 EDUCD                    24   
20              DEGFIELD                    37   
21             DEGFIELDD                   172   
22             DEGFIELD2                    37   
23            DEGFIELD2D                   171   
24               EMPSTAT                     1   
25              EMPSTATD                     1   
26              CLASSWKR                     2   
27             CLASSWKRD                     8   
28                OCCSOC                   525   
29              UHRSWORK                    65   
30               INCEARN                  3735   
31              TRANWORK                    12   
32              Industry                   262   
33              Category                    13   
34          Sub Category                    20   
35         Main Category                     5   
36   OCCSOC Sub Category                    23   
37  2018 Onward ACS/PRCS                   525   
38      Occupation title                   525   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Cleaning the data

### Fixing the categorical columns which should be numerical

In [30]:
data_final['NCHILD'] = data_final['NCHILD'].replace({'0 children present':0, '1 child present':1, '9+':9, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8})
data_final['NCHILD'] = pd.to_numeric(data_final['NCHILD'], errors='coerce')
data_final['NCHILD'].value_counts()

/var/folders/l5/9tzh8xvn4c9bb2qc2r3pkqtc0000gn/T/ipykernel_777/79927985.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_final['NCHILD'] = data_final['NCHILD'].replace({'0 children present':0, '1 child present':1, '9+':9, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8})


NCHILD
0    276522
1     97427
2     86119
3     30492
4      8497
5      2199
6       669
7       226
8       110
9       102
Name: count, dtype: int64

Now we first need to fix the year naturalization and citizenship

In [31]:
# We need to ensure if YRNATUR is not null, CITIZEN must be 'Naturalized citizen'
data_final.loc[data_final['YRNATUR'].notnull(), 'CITIZEN'] = 'Naturalized citizen'

# Fill missing values
data_final['CITIZEN'] = data_final['CITIZEN'].fillna('Unknown')
data_final['YRNATUR'] = data_final['YRNATUR'].fillna(9999)

In [32]:
data_final.loc[data_final['YRNATUR'].astype(str).str.len() > 6,'YRNATUR'] 

10120           1945 (1945-1947, 2017-onward ACS/PRCS)
30140           1948 (1948-1949, 2017-onward ACS/PRCS)
70731     1944 (1944 or earlier, 2019-onward ACS/PRCS)
111512          1948 (1948-1949, 2017-onward ACS/PRCS)
112605          1950 (1950-1951, 2020-onward ACS/PRCS)
127383          1945 (1945-1947, 2017-onward ACS/PRCS)
306192          1948 (1948-1949, 2017-onward ACS/PRCS)
317792          1948 (1948-1949, 2017-onward ACS/PRCS)
387446          1945 (1945-1947, 2017-onward ACS/PRCS)
478237          1950 (1950-1951, 2020-onward ACS/PRCS)
Name: YRNATUR, dtype: object

In [33]:
# Extract the first element after splitting each value in YRNATUR
data_final['YRNATUR'] = data_final['YRNATUR'].astype(str).str.split().str[0]

# Filter rows where the length is greater than 6
data_final.loc[data_final['YRNATUR'].str.len() > 6, 'YRNATUR']

Series([], Name: YRNATUR, dtype: object)

In [34]:
data_final['YRNATUR'] = pd.to_numeric(data_final['YRNATUR'])

Age needs to be fixed

In [35]:
data_final['AGE'] = pd.to_numeric(data_final['AGE'].astype(str).str.split().str[0])

Fixing UHRS work

In [36]:
data_final['UHRSWORK'] = pd.to_numeric(data_final['UHRSWORK'].astype(str).str.split().str[0])

In [37]:
data_final['UHRSWORK'].value_counts()

UHRSWORK
40    305362
50     53608
45     35094
60     23518
35     15628
55      9908
36      8599
48      6865
38      6146
42      4968
70      4351
37      4008
80      3585
65      2915
44      2057
52      1915
43      1877
46      1277
56      1239
39      1087
99       839
72       808
41       776
75       762
47       695
54       496
90       479
84       441
58       438
49       382
53       307
85       194
64       174
62       167
57       166
51       148
68       136
66       123
63       120
96       101
98        55
59        50
76        49
78        44
67        43
77        40
86        36
82        35
61        34
88        32
69        30
74        29
95        28
92        15
73        12
71        11
91        10
87        10
94         9
83         6
93         6
79         6
89         6
81         5
97         3
Name: count, dtype: int64

Fixing Incearn

In [39]:
display(find_non_numeric_values(data_final, 'INCEARN'))

{'$1 or breakeven': 2}

In [40]:
data_final['INCEARN'] = data_final['INCEARN'].replace({'$1 or breakeven':0})

## Creating a flag for female instead of SEX

In [46]:
data_final['SEX'] = data_final['SEX'].map({'Male': 0, 'Female': 0})
data_final = data_final.rename(columns={'SEX':'FEMALE_FLAG'}) 

In [47]:
data_final.head()

,REGION,STATEFIP,PERWT,NCHILD,FEMALE_FLAG,AGE,MARST,MARRNO,RACE,RACED,BPL,BPLD,ANCESTR1,ANCESTR1D,CITIZEN,YRNATUR,LANGUAGE,LANGUAGED,SPEAKENG,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,CLASSWKR,CLASSWKRD,OCCSOC,IND,WKSWORK1,UHRSWORK,INCTOT,INCWAGE,INCINVST,INCOTHER,INCEARN,TRANWORK,TRANTIME,DEPARTS,ARRIVES,IND Codes,Industry,Category,Sub Category,Main Category,OCCSOC Sub Category,2018 Onward ACS/PRCS,Occupation title
0,East South Central Div.,Alabama,48.0,0,0,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",Unknown,9999.0,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schools, including junior colleges","Educational Services, and Health Care and Social Assistance",Educational Services,SERVICE OCCUPATIONS:,Personal Care and Service Occupations:,399041,Residential Advisors
1,East South Central Div.,Alabama,9.0,0,0,52,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",Unknown,9999.0,English,English,"Yes, speaks only English",Grade 12,Regular high school diploma,NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",537051,6390,52,40,40600,40600,0,0,40600,"Auto, truck, or van",5,802,809,6390,Warehousing and storage,"Transportation and Warehousing, and Utilities","Transportation and Warehousing, and Utilities","PRODUCTION, TRANSPORTATION, AND MATERIAL MOVING OCCUPATIONS:",Material Moving Occupations:,537051,Industrial Truck and Tractor Operators
2,East South Central Div.,Alabama,46.0,0,0,18,Never married/single,Not Applicable,Black/African American,Black/African American,Alabama,Alabama,African-American,"African-American (1990-2000, ACS, PRCS)",Unknown,9999.0,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,State govt employee,412010,5391,52,41,6000,6000,0,0,6000,"Auto, truck, or van",25,532,559,5391,"General merchandise stores, including warehouse clubs and supercenters",Retail Trade,Retail Trade,SALES AND OFFICE OCCUPATIONS:,Sales and Related Occupations:,412010,Cashiers
3,East South Central Div.,Alabama,54.0,0,0,20,Never married/single,Not Applicable,Black/African American,Black/African American,Georgia,Georgia,African-American,"African-American (1990-2000, ACS, PRCS)",Unknown,9999.0,English,English,"Yes, speaks only English",1 year of college,"1 or more years of college credit, no degree",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",399041,7870,52,40,6300,6300,0,0,6300,"Auto, truck, or van",5,1915,1909,7870,"Colleges, universities, and professional schools, including junior colleges","Educational Services, and Health Care and Social Assistance",Educational Services,SERVICE OCCUPATIONS:,Personal Care and Service Occupations:,399041,Residential Advisors
4,East South Central Div.,Alabama,42.0,0,0,18,Never married/single,Not Applicable,Black/African American,Black/African American,Florida,Florida,African-American,"African-American (1990-2000, ACS, PRCS)",Unknown,9999.0,English,English,"Yes, speaks only English",Grade 12,"Some college, but less than 1 year",NaN,NaN,NaN,NaN,Employed,At work,Works for wages,"Wage/salary, private",352010,8680,52,35,26400,26400,0,0,26400,Walked only,3,1455,1459,8680,Restaurants and other food services,"Arts, Entertainment, and Recreation, and Accommodation and Food Services",Accommodation and Food Services,SERVICE OCCUPATIONS:,Food Preparation and Serving Related Occupations:,352010,Cooks


## Saving a non-encoding file

In [41]:
os.getcwd()

'/Users/vivan/Desktop/Central File Manager/03 Projects/03 Datathon/genderwagegap/01 Data'

In [48]:
#os.mkdir('01 Data Cleaned')
data_final.to_pickle(os.getcwd()+'/01 Data Cleaned/data_cleaned.pkl')

## Encoding
Now that we have cleaned the required categorical columns we should focus on encoding

In [45]:
# first we need to decide which files will be divided into frequency encoding and which go into one-hot and which will go into level encoding